In [36]:
from pyspark.sql import SparkSession

In [37]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

In [38]:
df_tweet = spark.read.json("/home/joao-santos/Desktop/academic_projects/alura/data_enginner/apache_airflow/datalake/silver/twitter_datascience/tweet")

In [39]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [40]:
from pyspark.sql import functions as f

In [41]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))

In [42]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2024-10-02|       8|  3186|   2970|   2022|     3510|    Wed|
|  2024-10-04|      27|  7510|   7405|   7585|     5670|    Fri|
|  2024-10-07|      10|  1302|    950|    748|     1058|    Mon|
|  2024-10-05|      34|  8380|   7300|   8064|     7484|    Sat|
|  2024-10-03|      19|  5016|   5280|   6264|     6762|    Thu|
|  2024-10-06|      17|  1732|   1898|   2514|     2520|    Sun|
+------------+--------+------+-------+-------+---------+-------+

